In [281]:
import glob
import pandas as pd
import numpy as np
from pathlib import Path
from itertools import product
import json

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [282]:
N_REPETITIONS = 10
N_ITERATIONS = 25
FPS_VALUES = [10, 12, 14, 15, 17, 20]

In [283]:
base_path = "data/ronin/rs"
paths = {"20 EPIC": f"{base_path}/*/checkpoints/it*/*20*True.json",
         "18 EPIC": f"{base_path}/*/checkpoints/it*/*18*True.json",
         "17 EPIC": f"{base_path}/*/checkpoints/it*/*17*True.json",
         "15 EPIC": f"{base_path}/*/checkpoints/it*/*15*True.json",
         "14 EPIC": f"{base_path}/*/checkpoints/it*/*14*True.json",
         "12 EPIC": f"{base_path}/*/checkpoints/it*/*12*True.json",
         "10 EPIC": f"{base_path}/*/checkpoints/it*/*10*True.json",

         "20 LOW": f"{base_path}/*/checkpoints/it*/*20*False.json",
         "10 LOW": f"{base_path}/*/checkpoints/it*/*10*False.json",

         }

for name, pattern in paths.items():
    n = len(glob.glob(pattern))
    print(
        f"For {name} there are {n}/{N_REPETITIONS*N_ITERATIONS} ready {n//N_ITERATIONS}")

For 20 EPIC there are 138/250 ready 5
For 18 EPIC there are 202/250 ready 8
For 17 EPIC there are 149/250 ready 5
For 15 EPIC there are 189/250 ready 7
For 14 EPIC there are 157/250 ready 6
For 12 EPIC there are 137/250 ready 5
For 10 EPIC there are 250/250 ready 10
For 20 LOW there are 177/250 ready 7
For 10 LOW there are 153/250 ready 6


In [284]:
def process_df(df):
    record_df = pd.json_normalize(df['records'])
    # concatenate records data
    df = pd.concat([df, record_df],  axis=1)

    # drop original column
    df = df.drop(['records', 'index'], axis=1)

    # drop infractions columns

    # remove prefixes from column name
    df.columns = df.columns.str.removeprefix('meta.')
    df.columns = df.columns.str.removeprefix('scores.')
    df['driving_score'] = df['score_composed'] / 100
    # df = df.rename(columns={"index": "route_index"})
    df = df.set_index(['fps', 'highquality', 'rep', 'it', 'route_id'])
    return df

In [285]:
def make_df(path: str):

    # DATA FROM PATH
    data = {}
    data['path'] = path
    stem = Path(path).stem.split("_")
    data['fps'] = int(stem[1])
    data['highquality'] = stem[3]
    match path.split("/"):
        case ["data", "ronin", "rs", rep, "checkpoints", it, *objects]:
            data['rep'] = int(rep)
            data['it'] = int(it[2:])
    # print(data)

    # READ CHECKPOINT
    with open(path, "r") as f:
        content = json.load(f)

        data['records'] = content['_checkpoint']['records']
        if not data['records']:
            return

        df = pd.DataFrame(data)
        return process_df(df)


dfs = []
pattern = f"{base_path}/*/checkpoints/it*/*.json"
for file in glob.glob(pattern):
    dfs.append(make_df(file))

In [289]:
df = pd.concat(dfs).sort_index()
df

path                      status                      infractions.collisions_layout infractions.collisions_pedestrian                     infractions.collisions_vehicle                    infractions.outside_route_lanes                              infractions.red_light infractions.route_dev infractions.route_timeout infractions.stop_infraction                        infractions.vehicle_blocked  duration_game  duration_system  route_length  score_composed  score_penalty  score_route  driving_score
fps highquality rep it route_id                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              
10  False       0   0  RouteScenario_585  data/ronin/rs/0/checkpoints/it0/fps_10_highqua...                   Completed                                                 []                                []                                                 []                                                 []                                                 []                    []                        []                          []                                                 []      99.800001       190.234782    521.709640      100.000000       1.000000   100.000000       1.000000
                    1  RouteScenario_33   data/ronin/rs/0/checkpoints/it1/fps_10_highqua...  Failed - Agent got blocked  [Agent collided against object with type=stati...                                []                                                 []  [Agent went outside its route lanes for about ...  [Agent ran a red light 341 at (x=186.95, y=226...                    []                        []                          []  [Agent got blocked at (x=186.444, y=309.387, z...     208.700003       453.360121    292.742151       14.697936       0.432891    33.952991       0.146979
                    2  RouteScenario_439  data/ronin/rs/0/checkpoints/it2/fps_10_highqua...    Failed - Agent timed out  [Agent collided against object with type=stati...                                []  [Agent collided against object with type=vehic...  [Agent went outside its route lanes for about ...  [Agent ran a red light 662 at (x=264.0, y=-302...                    []          [Route timeout.]                          []                                                 []     176.100003       450.602438    209.466124       21.555446       0.221831    97.170735       0.215554
                    3  RouteScenario_494  data/ronin/rs/0/checkpoints/it3/fps_10_highqua...                   Completed                                                 []                                []  [Agent collided against object with type=vehic...                                                 []                                                 []                    []                        []                          []                                                 []     121.100002       285.478132    607.780443       60.000000       0.600000   100.000000       0.600000
                    4  RouteScenario_591  data/ronin/rs/0/checkpoints/it4/fps_10_highqua...                   Completed                                                 []                                []                                                 []  [Agent went outside its route lanes for about ...                                                 []                    []                        []                          []                                                 []     104.200002       204.950902    395.800026       98.036363 

In [287]:
# Get an index of minimal driving score for each fidelity value for each repetition

solutions = df.groupby(
    ['fps', 'highquality', 'rep'],)['driving_score'].idxmin()
solutions

fps  highquality  rep
10   False        0      (10, False, 0, 21, RouteScenario_142)
                  1       (10, False, 1, 2, RouteScenario_476)
                  2      (10, False, 2, 16, RouteScenario_233)
                  3       (10, False, 3, 8, RouteScenario_150)
                  4      (10, False, 4, 19, RouteScenario_228)
                  5       (10, False, 5, 2, RouteScenario_533)
                  6       (10, False, 6, 1, RouteScenario_480)
     True         0        (10, True, 0, 2, RouteScenario_476)
                  1         (10, True, 1, 8, RouteScenario_11)
                  2        (10, True, 2, 13, RouteScenario_72)
                  3        (10, True, 3, 6, RouteScenario_277)
                  4        (10, True, 4, 2, RouteScenario_533)
                  5        (10, True, 5, 8, RouteScenario_419)
                  6       (10, True, 6, 23, RouteScenario_140)
                  7       (10, True, 7, 21, RouteScenario_691)
                  8        (10, T

In [288]:
solutions.size

67

^ should be 90, 10 for each fidelity value